In [1]:
!pip install yfinance

     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     ---------------------------------------- 10.2/948.2 kB ? eta -:--:--
     - ----------------------------------- 41.0/948.2 kB 393.8 kB/s eta 0:00:03
     --- --------------------------------- 92.2/948.2 kB 655.4 kB/s eta 0:00:02
     ----- ------------------------------ 143.4/948.2 kB 774.0 kB/s eta 0:00:02
     --------- ---------------------------- 225.3/948.2 kB 1.1 MB/s eta 0:00:01
     --------- ---------------------------- 225.3/948.2 kB 1.1 MB/s eta 0:00:01
     --------- ---------------------------- 225.3/948.2 kB 1.1 MB/s eta 0:00:01
     --------- ---------------------------- 225.3/948.2 kB 1.1 MB/s eta 0:00:01
     --------- ---------------------------- 225.3/948.2 kB 1.1 MB/s eta 0:00:01
     --------- ---------------------------- 225.3/948.2 kB 1.1 MB/s eta 0:00:01
     --------- ---------------------------- 225.3/948.2 kB 1.1 MB/s eta 0:00:01
     --------- ---------------------------- 225.3/948

In [25]:
import pandas as pd
import yfinance as yf
from datetime import datetime

# Step 1: Read the input CSV file (replace with your actual file path)
input_file = r"D:\alphanalysis assignment\Stocks.csv"  # Your input file path
stocks_df = pd.read_csv(input_file)

# Step 2: Validate date format
def validate_date_format(date_str):
    """
    Validate if the provided date string is in the correct format (YYYY-MM-DD).
    Returns None if invalid, else returns a datetime object.
    """
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        print(f"Invalid date format: {date_str}. Please use YYYY-MM-DD.")
        return None

# Step 3: Fetch stock data for the specified ticker and date range
def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch stock data for a specific ticker and date range using Yahoo Finance.
    Returns the stock data containing the 'Close' price.
    """
    try:
        # Fetch the data for the stock ticker and date range
        stock_data = yf.download(f"{ticker}.NS", start=start_date, end=end_date, progress=False)
        if stock_data.empty:
            print(f"No data found for {ticker}")
            return None
        return stock_data[['Close']]  # Only return the closing prices
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Step 4: Generate the final output with investments calculation
def generate_output(df, start_date, end_date, investment_amount):
    """
    Generate a DataFrame containing the investment calculation for each stock.
    """
    results = []  # List to store results for each ticker

    # Iterate through each row in the stock DataFrame
    for index, row in df.iterrows():
        ticker = row['Ticker']
        weightage = row['Weightage']
        
        # Fetch stock data for the specified dates
        stock_data = fetch_stock_data(ticker, start_date, end_date)
        if stock_data is None:
            continue
        
        # Filter the data to get only the start and end date prices
        stock_data = stock_data.loc[start_date:end_date]
        
        if stock_data.empty:
            print(f"No data found for {ticker} in the specified date range.")
            continue
        
        # Create a row for the ticker with weightage
        row_data = {'Ticker': ticker, 'Weightage': weightage}
        
        # Calculate the number of shares for start and end dates
        for date in stock_data.index:
            closing_price = stock_data.loc[date, 'Close']
            
            # Ensure closing_price is a scalar value (not a series)
            if isinstance(closing_price, pd.Series):
                closing_price = closing_price.iloc[0]
            
            # Calculate the amount to be invested based on the weightage
            amount_invested = weightage * investment_amount
            
            # Calculate the number of shares that can be bought with the invested amount
            if closing_price > 0:  # Ensure the closing price is greater than 0
                shares = amount_invested / closing_price
            else:
                shares = 0  # Handle cases where the price is zero to avoid division errors
            
            # Add shares data to the row for that date
            row_data[date.strftime('%Y-%m-%d')] = round(shares, 6)  # Round shares to 6 decimal places
        
        results.append(row_data)
    
    # Create a DataFrame for the final result
    result_df = pd.DataFrame(results)
    return result_df

# Step 5: Get start and end dates from user input with validation
start_date_str = input("Enter the start date (YYYY-MM-DD): ")
end_date_str = input("Enter the end date (YYYY-MM-DD): ")

start_date = validate_date_format(start_date_str)
end_date = validate_date_format(end_date_str)

# Validate if the dates are correct and start date is before end date
if start_date and end_date:
    if start_date > end_date:
        print("Error: Start date cannot be later than end date.")
    else:
        # Step 6: Get investment amount from user input
        try:
            investment_amount = float(input("Enter the total investment amount: "))
        except ValueError:
            print("Invalid input! Please enter a valid numeric value for the investment amount.")
            exit(1)
        
        # Step 7: Generate the output
        print("\nFetching stock data and generating output...\n")
        final_data = generate_output(stocks_df, start_date, end_date, investment_amount)

        # Step 8: Display the result
        print("\nGenerated Output DataFrame:\n")
        print(final_data)

        # Step 9: Save the results to Excel
        final_data.to_excel('stock_prices_output.xlsx', index=False)
        print("\nResults saved to 'stock_prices_output.xlsx'.")


Enter the start date (YYYY-MM-DD):  2023-01-03
Enter the end date (YYYY-MM-DD):  2023-01-05
Enter the total investment amount:  100000



Fetching stock data and generating output...


Generated Output DataFrame:

        Ticker  Weightage  2023-01-03  2023-01-04
0   ADANIPORTS     0.0082    0.999452    1.012346
1   APOLLOHOSP     0.0061    0.135830    0.137595
2   ASIANPAINT     0.0195    0.643936    0.646370
3     AXISBANK     0.0257    2.670685    2.684213
4   BAJAJFINSV     0.0120    0.767754    0.775269
5   BAJFINANCE     0.0237    0.358751    0.360676
6   BHARTIARTL     0.0233    2.849804    2.870165
7         BPCL     0.0046    2.731186    2.735653
8    BRITANNIA     0.0052    0.122588    0.122786
9        CIPLA     0.0068    0.632029    0.637361
10   COALINDIA     0.0051    2.275770    2.348607
11    DIVISLAB     0.0077    0.226938    0.224127
12     DRREDDY     0.0067    0.790402    0.787587
13   EICHERMOT     0.0049    0.152484    0.152399
14      GRASIM     0.0085    0.497003    0.504869
15     HCLTECH     0.0153    1.469247    1.475979
16    HDFCBANK     0.0810    4.940983    5.030900
17    HDFCLIFE     0.00